In [4]:
import numpy as np
import pickle
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

plt.style.use("ggplot")


In [5]:
import json



In [22]:
df = pd.read_excel('../data/Data_2020_P06/all_billings_inputs_08_20.xlsx',
                  sheet_name = 'base_billings')

In [23]:
df.rename(
        index=str,
        columns={
            "Contrct Duration in Months": "duration",
            "Document Currency": "curr",
            "Enterprise BU Desc": "BU",
            "Invoice Fiscal Year Period Desc": "period",
            "POB Type": "POB_type",
            "Product Config Type": "config",
            "Rev Rec Category": "rev_req_type",
            "Rule For Bill Date": "rebill_rule",
            "Sales Document Type": "sales_doc",
            "Sales Type": "sales_type",
            "Subscription Term": "sub_term",
            "Completed Sales ( DC )": "DC_amount",
            "Completed Sales": "US_amount",
        },
        inplace=True,
    )


In [24]:
df.head(3)

,duration,curr,BU,period,POB_type,config,rev_req_type,rebill_rule,sales_doc,sales_type,sub_term,DC_amount,US_amount
0,0,AUD,Experience Cloud,2017-01,NaN,OCONS,NaN,NaN,ZCC,PRO-SVC-INV,0,375.0,278.01
1,0,AUD,Experience Cloud,2017-01,NaN,ONORE,NaN,NaN,ZCC,RECOGNIZED,0,0.0,45.75
2,0,AUD,Experience Cloud,2017-01,NaN,ONORE,D,YA,ZCC,DEFERRED,0,-684738.0,-512177.81


In [25]:
vc = df["curr"].value_counts()
keep_these = vc.values > 20
keep_curr = vc[keep_these]
list_keepers = keep_curr.index
remove_these = vc[vc.values <= 20].index
model_dict = {"curr_removed": list(vc[remove_these].index)}
delete_curr = list(remove_these)

if "TRY" not in model_dict["curr_removed"]:
    model_dict["curr_removed"].append("TRY")
    delete_curr.append("TRY")
    list_keepers = list_keepers.drop("TRY")

df = df[df["curr"].isin(list_keepers)]

In [26]:
print(len(df))
df = df[df["DC_amount"] != 0]
print(len(df))

293832
286744


## Grouping by POB Type

The new 606 has everything grouped by sales type. There are blanks
We need to separate into groups based on the POB type


In [30]:
list_IR = ['IR', 'IR-NA', 'LFB']
list_service = ['CR', 'CR-NA']
list_deferred = ['RR', 'RR-NA']
list_hybrid = ['BNDL']

list_all = list_IR + list_service + list_deferred + list_hybrid
print(list_all)

['IR', 'IR-NA', 'LFB', 'CR', 'CR-NA', 'RR', 'RR-NA', 'BNDL']


In [33]:
rec = df[df["POB_type"].isin(list_IR)].copy()
svc = df[df["POB_type"].isin(list_service)].copy()
dfr = df[df["POB_type"].isin(list_deferred)].copy()
hyb = df[df["POB_type"].isin(list_hybrid)].copy()
blank = df[~df["POB_type"].isin(list_all)].copy()

In [37]:
print('rec', len(rec))
print('svc', len(svc))
print('dfr', len(dfr))
print('hyb', len(hyb))
print('blank', len(blank))
print('ALL', len(df))

rec 12299
svc 3058
dfr 75564
hyb 3604
blank 192219
ALL 286744


In [39]:
len(rec)+len(svc)+ len(dfr)+ len(blank)+len(hyb)

286744

In [40]:
gb_rec = rec.groupby(["curr", "BU", "period"], as_index=False).sum()

In [42]:
gb_rec.drop(labels=["duration", "sub_term"] , axis=1, inplace=True)

In [43]:
gb_rec.head(4)

,curr,BU,period,DC_amount,US_amount
0,ARS,Creative,2019-07,6786.00,155.12
1,ARS,Creative,2019-08,16472.00,390.02
2,ARS,Creative,2019-09,19205.00,405.51
3,ARS,Creative,2019-10,30382.19,532.13


In [44]:
gb_svc = svc.groupby(["curr", "BU", "period"], as_index=False).sum()

In [ ]:
gb_svc

In [45]:
gb_svc.head(4)

,curr,BU,period,duration,sub_term,DC_amount,US_amount
0,AUD,Creative,2017-10,18,0,8544.00,6837.59
1,AUD,Creative,2019-01,12,0,8472.55,6135.45
2,AUD,Creative,2019-02,12,0,3635.13,2587.61
3,AUD,Creative,2019-03,36,24,10437.33,7531.36


# Deferred billings
## Type B Service based/

In [46]:
dfr_b = dfr[dfr["rev_req_type"] == "B"].copy()

In [47]:
print(len(dfr))
print(len(dfr_b))

75564
0


In [49]:
dfr.head(10)

,duration,curr,BU,period,POB_type,config,rev_req_type,rebill_rule,sales_doc,sales_type,sub_term,DC_amount,US_amount
15,0,EUR,Experience Cloud,2017-01,RR,ONORE,D,YA,ZCC,DEFERRED,0,117600.00,123853.97
47,1,EUR,Experience Cloud,2017-01,RR,ONORE,D,YA,ZCC,DEFERRED,0,399.00,426.24
48,1,JPY,Experience Cloud,2017-01,RR,ONORE,D,YA,ZCC,DEFERRED,0,72772.00,637.48
55,1,USD,Experience Cloud,2017-01,RR,ONORE,D,Y1,ZCC,DEFERRED,0,1232.50,1232.50
56,1,USD,Experience Cloud,2017-01,RR,ONORE,D,YA,ZCC,DEFERRED,0,18094.74,18094.74
57,1,USD,Experience Cloud,2017-01,RR,ONORE,D,YQ,ZCC,DEFERRED,0,4064.52,4064.52
60,2,EUR,Experience Cloud,2017-01,RR,ONORE,D,YA,ZCC,DEFERRED,0,19485.90,20724.03
65,2,USD,Experience Cloud,2017-01,RR,ONORE,D,YA,ZCC,DEFERRED,0,13850.00,13850.00
69,3,CAD,Experience Cloud,2017-01,RR,ONORE,D,YQ,ZCC,DEFERRED,0,78810.00,58634.64
74,3,EUR,Experience Cloud,2017-01,RR,ONORE,D,YA,ZCC,DEFERRED,0,79251.79,84287.45


In [50]:
dfr.rev_req_type.value_counts()

D    51470
A    23504
Name: rev_req_type, dtype: int64

## Type A Deferred billings

### First sort by config type

### Then use sub term

In [55]:
dfr_a = dfr[dfr["rev_req_type"] == "A"].copy()
print("length of dfr_a", len(dfr_a))
dfr_a.drop(labels='duration', axis=1, inplace=True)
gb_a = dfr_a.groupby(["curr", "BU", "period", "config", 'sub_term'], as_index=False).sum()


length of dfr_a 23504


In [56]:
len(gb_a)

2744

In [58]:
gb_a.head(4)


,curr,BU,period,config,sub_term,DC_amount,US_amount
0,AUD,Creative,2017-01,1Y,12,52763.75,39147.16
1,AUD,Creative,2017-01,2Y,0,562.00,417.19
2,AUD,Creative,2017-01,2Y,12,0.00,0.00
3,AUD,Creative,2017-02,1Y,0,11438.00,8498.43


In [59]:
gb_a.config.value_counts()

1Y       1757
2Y        664
OUNIV     112
3Y        102
MTHLY      82
OCONS      14
ONORE      13
Name: config, dtype: int64

In [60]:
gb_a.sub_term.value_counts()

0     1524
12     927
1      184
36     102
24       7
Name: sub_term, dtype: int64